<a href="https://colab.research.google.com/github/wellicfaria/analise_dados/blob/main/Execicio_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercicio e Analise de Dados. 

Nome(s): 

- Wellington Faria
- Thais Nabe

------------------------------------------------------------------------------------------------------------------------------

O Relatório deve apresentar as seguintes informações:

Base de Dados Utilizada: Base Logistica.xlsx

#### Identificar:

- Identificar os problemas pelas Dimensão de QD (ou seja, dado um problema, qual a dimensão de QD que descreve o problema?);

- exemplificar o problema para cada dimensão de QD identificada, usando os dados da base de dados;

- indicar o percentual de dados que apresenta cada um dos problemas identificados; 

### Intervenções realizadas

- Para cada problema/dimensão de QD, mostrar que tratamento foi aplicado aos dados;
- Custo 
- Tente monetizar o trabalho que foi realizado (mostre como calculou o custo).

In [ ]:
!pip install pyspark

# Dados a serem analinados

Dados de logistica.

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count, lit, datediff

In [ ]:
spark = SparkSession.builder.appName("APP_LOGISTICA").getOrCreate()

pdf = pd.read_excel('/content/Base Logistica.xlsx', index_col=0)
pdf.to_csv('/content/Base_Logistica.csv',index=False, header=True)
df = spark.read.csv('/content/Base_Logistica.csv', header=True, inferSchema=True)
df.show()
total_registros = df.count()
print(f'QUANTIDADE DE REGISTRO NA BASE: {total_registros}')

+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|        Nome Cliente| Município Início|Município Destino|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|      Caramuru Pie I|Rio Grande do Sul|           Santos|  1400.0|    2018-01-02|     2018-01-04|          3.0|          2.0|
|              Indaia|   Belo Horizonte|   Rio De Janeiro|   450.0|    2018-01-04|     2018-01-10|          8.0|          6.0|
|        Diamante Iii|           Santos|   Rio De Janeiro|   500.0|    2018-01-04|     2018-01-08|         10.0|          4.0|
|        Diamante Iii|           Santos|          Vitória|  1100.0|    2018-01-04|     2018-01-14|          6.0|         10.0|
|                null|   Belo Horizonte|           Santos|   500.0|    2018-01-04|     2018-01-14|          2.0

## Dimensão Completude (completeness)

### Análise (completeness): 

Identificamos problema de completude em:

- **Nome Cliente**: 46 ocorrencias, 1.68% da base
- **Total km**: 1 ocorrencia, 0.0038% da base
- **Prazo Entrega**: 20 ocorrencias, 0,073% da base
- **Tempo Entrega**: 26 ocorrencias, 0,95% da base

### Quantidade de Nulls: 

In [ ]:
# Quantidade de Nulos
df_null = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   )

df_null.show()

+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|Nome Cliente|Município Início|Município Destino|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|          46|               0|                0|       1|             0|              0|           20|           26|
+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+



### Percentual de Nulls: 

In [ ]:
# Porcentagem de Nulos 
df_null_percente = df_null.select([ (col(c)/lit(total_registros)*100).alias(c) for c in df.columns])
df_null_percente.show()

+------------------+----------------+-----------------+-------------------+--------------+---------------+------------------+------------------+
|      Nome Cliente|Município Início|Município Destino|           Total Km|Data do Pedido|Data da Entrega|     Prazo Entrega|     Tempo Entrega|
+------------------+----------------+-----------------+-------------------+--------------+---------------+------------------+------------------+
|1.6880733944954127|             0.0|              0.0|0.03669724770642202|           0.0|            0.0|0.7339449541284404|0.9541284403669725|
+------------------+----------------+-----------------+-------------------+--------------+---------------+------------------+------------------+



### Invervensão Realizada (completeness)

- **Tempo Entrega**:  Completamos os null de tempo de entrega com valor da subtração Data da Entrega e Data do Pedido.
- **Prazo Entrega**: Para prazo de entrega, consideramos Prazo Entrega = Tempo Entrega, para completar os dados null. Sendo assim, consiremos que pedidos foram entrague no ultimo dia do prazo.
- **Total Km**: Para completar os null do Total Km consideramosa media entre 'Município Início' e 'Município Destino', completando com essa media calculada
- **Nome Cliente**: Nao foi possivel identificar o cliente, pois a analise, portanto colocamos nos valosres null a string "Não Identificado"

#### Tempo Entrega

In [ ]:
# Analise Tempo Entrega:
#  - Analisando os null, podemos calcular o o tempo de entrega pela subtração de Data da Entrega e Data do Pedido

print('DATA FRAME COM TEMPO ENTREGA NULL')
func_null = isnan(col('Tempo Entrega')) | (col('Tempo Entrega').isNull())
df_tempo_entrega_null = df.filter(func_null)
df_tempo_entrega_null.show(30)

# TRATAMENTO
df = df.withColumn('Tempo Entrega', 
                   when(func_null, datediff(col('Data da Entrega').cast('date'), col('Data do Pedido').cast('date'))).\
                   otherwise(col('Tempo Entrega'))
                   )

DATA FRAME COM TEMPO ENTREGA NULL
+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|        Nome Cliente| Município Início|Município Destino|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|       Tempo Energia|   Belo Horizonte|          Vitória|   400.0|    2018-01-06|     2018-01-07|         null|         null|
|         Yara Brasil|          Vitória|           Santos|  1100.0|    2018-01-11|     2018-01-17|         null|         null|
|   Ind De Moveis Thb|          Vitória|Rio Grande do Sul|  2000.0|    2018-01-12|     2018-01-14|          4.0|         null|
|         Terphane Ne|Rio Grande do Sul|   Belo Horizonte|  2000.0|    2018-01-16|     2018-01-23|         null|         null|
|         Unimed Sjrp|           Santos|   Belo Horizonte|   500.0|    2018-0

#### Prazo Entrega

In [ ]:
# Analise Prazo Entrega: 
# - Para completar Prazo de entrega, consideramos que todas entragas foram feiras no ultimo dia. 
# - ou seja, tempo Prazo Entrega = Tempo Entrega
func_null = isnan(col('Prazo Entrega')) | (col('Prazo Entrega').isNull())
df_prazo_entrega_null = df.filter(func_null)
df_prazo_entrega_null.show(30)

# TRATAMENTO
df = df.withColumn('Prazo Entrega', 
                   when(func_null, col('Tempo Entrega')).\
                   otherwise(col('Prazo Entrega'))
                   )

+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|        Nome Cliente| Município Início|Município Destino|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+--------------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|       Tempo Energia|   Belo Horizonte|          Vitória|   400.0|    2018-01-06|     2018-01-07|         null|          1.0|
|         Yara Brasil|          Vitória|           Santos|  1100.0|    2018-01-11|     2018-01-17|         null|          6.0|
|         Terphane Ne|Rio Grande do Sul|   Belo Horizonte|  2000.0|    2018-01-16|     2018-01-23|         null|          7.0|
|           Xavantina|Rio Grande do Sul|          Niterói|  1600.0|    2018-01-25|     2018-01-29|         null|          4.0|
|         Agro Latina|           Santos|        São Paulo|   100.0|    2018-02-02|     2018-02-03|         null

#### Total Km

In [ ]:
# Analise Total Km
# Notamos que so temos um registro com Total Km
# Vamos completar os null com media considerando 'Município Início' e 'Município Destino'
# No caso, vimos que media de Rio De Janeiro a Niterói é 20 km. Arrumando esse registro. 

func_null = isnan(col('Total Km')) | (col('Total Km').isNull())
df_total_km_null = df.filter(func_null)

print('DATAFRAME CONTANDO TOTAL KML NULOS')
df_total_km_null.show(30)


df_media_km = df.select(['Município Início', 'Município Destino', 'Total Km']).\
  groupBy(['Município Início', 'Município Destino']).\
  agg({'Total Km':'mean'})

print('DATA FRAME COM A MEDIA')

df_join_media_total_km = df.join(df_media_km, ['Município Início', 'Município Destino'], 'left')
df_join_media_total_km.filter(func_null).show()


df = df_join_media_total_km.withColumn(
    'Total Km',
    when(func_null, col('avg(Total Km)')).\
    otherwise(col('Total Km'))
    
).drop('avg(Total Km)')

DATAFRAME CONTANDO TOTAL KML NULOS
+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|Nome Cliente|Município Início|Município Destino|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+
|Diamante Iii|  Rio De Janeiro|          Niterói|    null|    2018-06-08|     2018-06-09|         10.0|          1.0|
+------------+----------------+-----------------+--------+--------------+---------------+-------------+-------------+

DATA FRAME COM A MEDIA
+----------------+-----------------+------------+--------+--------------+---------------+-------------+-------------+-------------+
|Município Início|Município Destino|Nome Cliente|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|avg(Total Km)|
+----------------+-----------------+------------+--------+--------------+--------------

#### Nome Cliente 

In [ ]:
# Analise Nome Cliente 
# - Fazendo uma analise, identificamos que nao é possivel identificar cliente. 
# - Assim, tratamos o dado de null pela string "Nao Identificado"

func_null = isnan(col('Nome Cliente')) | (col('Nome Cliente').isNull())
df_nome_cliente_null = df.filter(func_null)

df_nome_cliente_null.show()

df.filter((col('Município Início')==lit('Vitória')) & (col('Município Destino')==lit('Niterói'))).show()

df = df.withColumn('Nome Cliente', 
                   when(func_null, lit('Não Identificado')).\
                   otherwise(col('Nome Cliente'))
                   )

+----------------+-----------------+------------+--------+--------------+---------------+-------------+-------------+
|Município Início|Município Destino|Nome Cliente|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+----------------+-----------------+------------+--------+--------------+---------------+-------------+-------------+
|  Belo Horizonte|           Santos|        null|   500.0|    2018-01-04|     2018-01-14|          2.0|         10.0|
|          Santos|           Santos|        null|     4.0|    2018-01-05|     2018-01-10|          8.0|        300.0|
|  Rio De Janeiro|           Santos|        null|   500.0|    2018-01-05|     2018-01-14|          6.0|          9.0|
|         Vitória|   Rio De Janeiro|        null|   500.0|    2018-01-09|     2018-01-10|          7.0|          1.0|
|  Belo Horizonte|          Niterói|        null|   450.0|    2018-01-24|     2018-01-31|          9.0|          7.0|
|  Belo Horizonte|          Niterói|        null|   450.

## Dimensão Consistência (consistency)

### Analise (consistency)

- **Tempo Entrega**: 2 casos de Consitencia, 0.073% da base. 
- **Total Km**: Os trajetos de *Rio Grande do Sul -> Niterói* e *Vitória -> Niterói* apresentaram dados inconsistentes. Totalizando 2 registros, 0.073% da base.



#### Tempo Entrega

In [ ]:
# Tempo Entrega
# - Tempo de entrega deve ser igual a diferenca de dias entre Data da Entrega e Data do Pedido

df_tempo_entrega_consistency = df.withColumn('diff', datediff(col('Data da Entrega').cast('date'), col('Data do Pedido').cast('date')))

df_tempo_entrega_consistency = df_tempo_entrega_consistency.filter(col('Tempo Entrega')!=col('diff'))
df_tempo_entrega_consistency.show()

print(f'PECENTUAL DA BASE {df_tempo_entrega_consistency.count()/total_registros*100}%')

+----------------+-----------------+----------------+--------+--------------+---------------+-------------+-------------+----+
|Município Início|Município Destino|    Nome Cliente|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|diff|
+----------------+-----------------+----------------+--------+--------------+---------------+-------------+-------------+----+
|          Santos|           Santos|Não Identificado|     4.0|    2018-01-05|     2018-01-10|          8.0|        300.0|   5|
|         Niterói|          Vitória|         Candura|   500.0|    2019-08-02|     2019-08-08|         30.0|         30.0|   6|
+----------------+-----------------+----------------+--------+--------------+---------------+-------------+-------------+----+

PECENTUAL DA BASE 0.07339449541284404%


#### Total Km

In [ ]:
# Total Km
# - Fazendo uma analise pelo skewness, podemos ver que temos alguns registros estao tendo outliers
# - Esse outliers, sao nos trajetos Rio Grande do Sul para Niterói e   Vitória e Niterói.

df_max = df.select(['Município Início','Município Destino', 'Total Km']).\
  groupBy(['Município Início','Município Destino']).\
  agg({'Total Km':'max'})

df_min = df.select(['Município Início','Município Destino', 'Total Km']).\
  groupBy(['Município Início','Município Destino']).\
  agg({'Total Km':'min'})


df_mean = df.select(['Município Início','Município Destino', 'Total Km']).\
  groupBy(['Município Início','Município Destino']).\
  agg({'Total Km':'mean'})

df_skewness = df.select(['Município Início','Município Destino', 'Total Km']).\
  groupBy(['Município Início','Município Destino']).\
  agg({'Total Km':'skewness'})

df_count = df.select(['Município Início','Município Destino', 'Total Km']).\
  groupBy(['Município Início','Município Destino']).\
  agg({'Total Km':'count'})

regra = ['Município Início', 'Município Destino']

print('DATAFRAME COM AS METRICAS')
df_max.join(df_min,regra).join(df_mean,regra).join(df_skewness,regra).join(df_count,regra).\
  filter(col('skewness(Total Km)').isNotNull()).\
  filter((col('skewness(Total Km)')>1.0) | (col('skewness(Total Km)')<-1.0)).\
  show()


print('Rio Grande do Sul -> Niterói')
df_t1 = df.filter( (col('Município Destino')=='Niterói') & (col('Município Início')=='Rio Grande do Sul'))
df_t1.show(5)

print(f'VALOR QUE APARECE MAIS Rio Grande do Sul -> Niterói: {df_t1.approxQuantile("Total Km", [0.5], 0.25)}')

print('COUNT DOS VALORES')
df_t1.select(['Total Km']).\
  groupBy(['Total Km']).\
  agg({'Total Km':'count'}).show()


print('Vitória -> Niterói')
df_t2 = df.filter( (col('Município Destino')=='Niterói') & (col('Município Início')=='Vitória'))
df_t2.show(5)

print(f'VALOR QUE APARECE MAIS Vitória -> Niterói: {df_t2.approxQuantile("Total Km", [0.5], 0.25)}')


print('COUNT DOS VALORES')
df_t2.select(['Total Km']).\
  groupBy(['Total Km']).\
  agg({'Total Km':'count'}).show()

DATAFRAME COM AS METRICAS
+-----------------+-----------------+-------------+-------------+-------------+------------------+---------------+
| Município Início|Município Destino|max(Total Km)|min(Total Km)|avg(Total Km)|skewness(Total Km)|count(Total Km)|
+-----------------+-----------------+-------------+-------------+-------------+------------------+---------------+
|Rio Grande do Sul|          Niterói|     200000.0|       1600.0|       5568.0| 6.857142857142857|             50|
|          Vitória|          Niterói|        500.0|          0.0|     492.1875|-7.811265775524034|             64|
+-----------------+-----------------+-------------+-------------+-------------+------------------+---------------+

Rio Grande do Sul -> Niterói
+-----------------+-----------------+-----------------+--------+--------------+---------------+-------------+-------------+
| Município Início|Município Destino|     Nome Cliente|Total Km|Data do Pedido|Data da Entrega|Prazo Entrega|Tempo Entrega|
+-----

### Invervensão Realizada (consistency)

- **Tempo Entrega** : Para os dados que tinham inconsidencia, calculamos o tempo de entrega usando a data de pedido e entrega. 
- **Total Km**:  Foi encontrado 2 registro com outliers, os mesmo foram alterados pelo valor de maior frequencia para os trajetos. 
-- *Rio Grande do Sul -> Niterói* com Total Km com valor de 200000.0, foi alterado pelo valor 1600.0(conforme calculado na analise). 
-- *Vitória -> Niterói* com Total Km com valor de 0.0, foi alterado pelo valor 500.0(conforme calculado na analise). 



#### Tempo Entrega

In [ ]:
# Tempo Entrega
# O que deram inconsistencia calculamos 


df_aux = df.withColumn('diff', datediff(col('Data da Entrega').cast('date'), col('Data do Pedido').cast('date')))

#TRANSFORMACAO
df_aux = df_aux.withColumn('Tempo Entrega', 
                           when(col('Tempo Entrega')!=col('diff'), col('diff')).\
                           otherwise(col('Tempo Entrega'))
                           ).drop('diff')

df = df_aux

#### Total Km

In [ ]:
## Total Km
# Foi encontrado 2 registro com outliers. 

# - Rio Grande do Sul -> Niterói com Total Km com valor de 200000.0, sendo é 1600.0. 
# - Vitória -> Niterói com Total Km com valor de 0.0, sendo é 500.0. 

df = df.withColumn('Total Km', when(
    (col('Município Destino')=='Niterói') & (col('Município Início')=='Rio Grande do Sul'),lit(1600.0)
    ).otherwise(col('Total Km'))
)

df = df.withColumn('Total Km', when(
     (col('Município Destino')=='Niterói') & (col('Município Início')=='Vitória') ,lit(500.0)
    ).otherwise(col('Total Km'))
)



df_t1 = df.filter( (col('Município Destino')=='Niterói') & (col('Município Início')=='Rio Grande do Sul'))
df_t1.select(['Total Km']).\
  groupBy(['Total Km']).\
  agg({'Total Km':'count'}).show()


df_t2 = df.filter( (col('Município Destino')=='Niterói') & (col('Município Início')=='Vitória'))
df_t2.select(['Total Km']).\
  groupBy(['Total Km']).\
  agg({'Total Km':'count'}).show()



+--------+---------------+
|Total Km|count(Total Km)|
+--------+---------------+
|  1600.0|             50|
+--------+---------------+

+--------+---------------+
|Total Km|count(Total Km)|
+--------+---------------+
|   500.0|             64|
+--------+---------------+



## Analise Final


## Impactos Financeiros

- Identficamos quantidade de Total de KM, incompletos e com valores inconsitentes. Isso pode levar ao calculo incorreto dos custos, calculando os lucros incorretamente. 

- Foi identificado entregas sem o nome do cliente. Isso pode impactar na cobrança dos cliente, não sendo possivel receber o frete. 

- Os custos para tratamentos desses dados também gastaram homem hora e poder de processamento. O ideal é que esse dados sejam arrumados na fonte, evitando gastos de processamento para correção. 


## Custos de Análise

- Considerando o valor hora em 110 reais. Como o tempo gasto para análise e correção dos dodos foi de 20h, totalizando 2200 reais.



In [ ]:
pandasDF = df.toPandas()
pandasDF.to_excel('/content/Base_Logistica_Corrigida.xlsx', sheet_name='Sheet1',index=False)